# Amy Huynh : Assignment 2

__Due date__: April 25, 2018 at 10 pm
__Submission__: IPython notebook to GauchoSpace

We have been building up to creating components for a dashboard app in Jupyter notebook:

- Data downloading function: `get_nba_data()` in a `03-Data-collection-and-manipulation.ipynb`. 
- Accessing pandas data frame and creating dictionary structures in `04-Pandas-Data-Frame.ipynb` and `05-Data-Frame-and-Visualization.ipynb`.
- Creating widgets for interactivity: `05-Data-Frame-and-Visualization.ipynb`.
- Plotting visualizations with Matplotlib and Seaborn: `05-Data-Frame-and-Visualization.ipynb`.

We can put these components together to create an interactive dashboard for creating something similar to the R package: https://github.com/toddwschneider/ballr

Your assignment is to create an interactive dashboard.

It doesn't have to be exactly the same as the package or what I have proposed. If you would like to create another visualization, that would be great as well. I will refer to __default option__ as continuing to do what we started in class: dashboard of shotchart similar to what [BallR package](https://github.com/toddwschneider/ballr) does. I will refer to __open-ended option__ as creating a dashboard of your choice.

Below, I specify some necessary components to your dashboard.

## Problem 1: Data Download

__Default option__: you can use the `get_nba_data()` function. No additional work is needed.

In [ ]:
# from Lecture Four:

import pandas as pd

def get_nba_data(endpt, params, return_url=False):

    ## endpt: https://github.com/seemethere/nba_py/wiki/stats.nba.com-Endpoint-Documentation
    ## params: dictionary of parameters: i.e., {'LeagueID':'00'}
    
    from pandas import DataFrame
    from urllib.parse import urlencode
    import json
    
    useragent = "\"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9\""
    dataurl = "\"" + "http://stats.nba.com/stats/" + endpt + "?" + urlencode(params) + "\""
    
    # for debugging: just return the url
    if return_url:
        return(dataurl)
    
    jsonstr = !wget -q -O - --user-agent={useragent} {dataurl}
    
    data = json.loads(jsonstr[0])
    
    h = data['resultSets'][0]['headers']
    d = data['resultSets'][0]['rowSet']
    
    return(DataFrame(d, columns=h))

In [ ]:
# Getting the Teams:

params = {'LeagueID':'00'}
teams = get_nba_data('commonTeamYears', params)

teams

In [ ]:
# Getting the Players:

params = {'LeagueID':'00', 'Season': '2016-17', 'IsOnlyCurrentSeason': '0'}
players = get_nba_data('commonallplayers', params)

players

## Problem 2: Creating interactive widgets & Problem 3: Downloading data with changing widget states

combined problems 2 and 3 into one area // created interactive widgets to auto fetch data for us using the 'observe' and 'on_click' functions

In [ ]:
team_names = players[['TEAM_ABBREVIATION', 'TEAM_NAME']].drop_duplicates()#.set_index('TEAM_ABBREVIATION')
team_names.head()

In [ ]:
teams_full = pd.merge(teams, team_names, left_on='ABBREVIATION', right_on='TEAM_ABBREVIATION')

teams_full = teams_full.sort('MIN_YEAR')
teams_full

In [ ]:
start_of_team_text = teams_full.MIN_YEAR
start_of_team_text = start_of_team_text
#start_of_team_text

start_of_team_dd = dict(zip(start_of_team_text, teams_full.MIN_YEAR))
start_of_team_dd

In [ ]:
team_dd = dict()

for t, p in teams_full.groupby('MIN_YEAR'):
    
    team_dd[t] = dict(zip(p.TEAM_ABBREVIATION + ', ' + p.TEAM_NAME, p.TEAM_ID))
team_dd

In [ ]:
player_dd = dict()

for t, p in players.groupby('TEAM_ID'):
    player_dd[t] = dict(zip(p.DISPLAY_FIRST_LAST, p.PERSON_ID))

player_dd[1610612765]
player_dd

In [ ]:
# Building Widgets:
from ipywidgets import interact, FloatSlider, Dropdown, Button

select = '1976'

start_menu= Dropdown(options = start_of_team_dd, label = select)
team_menu = Dropdown(options = team_dd[start_of_team_dd[select]])

#team_value = team_menu.value
player_menu = Dropdown(options=player_dd[team_menu.value])

fetch_button = Button(description='Get Data!', icon='check')

display(start_menu, team_menu, player_menu, fetch_button)


def switcharoo(change):
    team_menu.index = None
    team_menu.options = team_dd[change['new']]

def switcharoo2(change):
    player_menu.index = None
    if team_menu.value is not None:
        player_menu.options = player_dd[team_menu.value]

start_menu.observe(switcharoo, names = 'value')  
team_menu.observe(switcharoo2, names = 'value')

def get_data(change):
    print(start_menu.value, team_menu.value, player_menu.value)
    
fetch_button.on_click(get_data)

In [ ]:
# Creating another function to easily fetch the desired data for us:
def fetch_data(change):
    params = {'PlayerID':player_menu.value,
          'PlayerPosition':'',
          'Season':'2016-17',
          'ContextMeasure':'FGA',
          'DateFrom':'',
          'DateTo':'',
          'GameID':'',
          'GameSegment':'',
          'LastNGames':'0',
          'LeagueID':'00',
          'Location':'',
          'Month':'0',
          'OpponentTeamID':'0',
          'Outcome':'',
          'Period':'0',
          'Position':'',
          'RookieYear':'',
          'SeasonSegment':'',
          'SeasonType':'Regular Season',
          'TeamID': team_menu.value,
          'VsConference':'',
          'VsDivision':''}
    shot_stats = get_nba_data('shotchartdetail', params)
    print (shot_stats.head(10)) # show only the first 10 rows for now

fetch_button2 = Button(description='Get Data!', icon='check')
display(fetch_button2)

fetch_button2.on_click(fetch_data)

## Problem 4: Data transformation and visualization

__Default option__: create at least two data transformation using split-apply-combine approach. Some ideas are, 

- What is the shooting average against different teams? You would split based on opposing team, compute the average, and plot a bar chart. 

- What is the shooting average over different periods per game? You would split based on periods and game, then plot the changing shooting average over periods. Is this helpful? Why? Why not?

- Any other setting in which you would need to split-apply-combine to calculate a summary statistic.

- A setting of your choosing

Plot your result

__Open-ended option__: create at least one data transformation using split-apply-combine approach.

Plot your result

#### Transformation (1): Player Shot Comparison
James Harden v.s. Stephen Curry

In [ ]:
# James Harden Shot Data:

## param values for team id and player id found from the drop down widget function created above

harden_params = {'PlayerID': '201935',
          'PlayerPosition':'',
          'Season':'2016-17',
          'ContextMeasure':'FGA',
          'DateFrom':'',
          'DateTo':'',
          'GameID':'',
          'GameSegment':'',
          'LastNGames':'0',
          'LeagueID':'00',
          'Location':'',
          'Month':'0',
          'OpponentTeamID':'0',
          'Outcome':'',
          'Period':'0',
          'Position':'',
          'RookieYear':'',
          'SeasonSegment':'',
          'SeasonType':'Regular Season',
          'TeamID': '1610612745',
          'VsConference':'',
          'VsDivision':''}

harden_shots = get_nba_data('shotchartdetail', harden_params)
#harden_shots.head(10)

for r, d in harden_shots.groupby('PERIOD'):
    print("***", r)
    print(d.head())

In [ ]:
# average minutes remaining per period for Harden
harden_shots.groupby('PERIOD')['MINUTES_REMAINING'].mean()

In [ ]:
# Stephen Curry Shot Data:
curry_params = {'PlayerID': '201939',
          'PlayerPosition':'',
          'Season':'2016-17',
          'ContextMeasure':'FGA',
          'DateFrom':'',
          'DateTo':'',
          'GameID':'',
          'GameSegment':'',
          'LastNGames':'0',
          'LeagueID':'00',
          'Location':'',
          'Month':'0',
          'OpponentTeamID':'0',
          'Outcome':'',
          'Period':'0',
          'Position':'',
          'RookieYear':'',
          'SeasonSegment':'',
          'SeasonType':'Regular Season',
          'TeamID': '1610612744',
          'VsConference':'',
          'VsDivision':''}

curry_shots = get_nba_data('shotchartdetail', curry_params)
#curry_shots.head(10)

In [ ]:
for r, d in curry_shots.groupby('PERIOD'):
    print("***", r)
    print(d.head())

In [ ]:
# average minutes remaining per period for Curry
curry_shots.groupby('PERIOD')['MINUTES_REMAINING'].mean()

In [ ]:
avg_min = {
    '1':[5.040909, 4.972281],
    '2': [2.355263, 2.631769],
    '3': [5.274633, 4.590164],
    '4': [3.434783, 3.461929],
    '5': [2.571429, 1.500000],
    '6': [1.250000, 4.000000]
}
index = ["Harden", "Curry"]
avg_min_df = pd.DataFrame(avg_min, index = index)
avg_min_df

In [ ]:
avg_min_df.plot.bar(figsize = (8, 6))

In [ ]:
# Half Court Graph from Lecture 05

%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

## code is from http://savvastjortjoglou.com/nba-shot-sharts.html
def draw_court(ax=None, color='black', lw=1, outer_lines=False):
    
    from matplotlib.patches import Circle, Rectangle, Arc
    from matplotlib.pyplot import gca
    
    # If an axes object isn't provided to plot onto, just get current one
    if ax is None:
        ax = gca()

    # Create the various parts of an NBA basketball court

    # Create the basketball hoop
    # Diameter of a hoop is 18" so it has a radius of 9", which is a value
    # 7.5 in our coordinate system
    hoop = Circle((0, 0), radius=7.5, linewidth=lw, color=color, fill=False)

    # Create backboard
    backboard = Rectangle((-30, -7.5), 60, 0, linewidth=lw, color=color)

    # The paint
    # Create the outer box 0f the paint, width=16ft, height=19ft
    outer_box = Rectangle((-80, -47.5), 160, 190, linewidth=lw, color=color,
                          fill=False)
    # Create the inner box of the paint, widt=12ft, height=19ft
    inner_box = Rectangle((-60, -47.5), 120, 190, linewidth=lw, color=color,
                          fill=False)

    # Create free throw top arc
    top_free_throw = Arc((0, 142.5), 120, 120, theta1=0, theta2=180,
                         linewidth=lw, color=color, fill=False)
    # Create free throw bottom arc
    bottom_free_throw = Arc((0, 142.5), 120, 120, theta1=180, theta2=0,
                            linewidth=lw, color=color, linestyle='dashed')
    # Restricted Zone, it is an arc with 4ft radius from center of the hoop
    restricted = Arc((0, 0), 80, 80, theta1=0, theta2=180, linewidth=lw,
                     color=color)

    # Three point line
    # Create the side 3pt lines, they are 14ft long before they begin to arc
    corner_three_a = Rectangle((-219, -47.5), 0, 140, linewidth=lw,
                               color=color)
    corner_three_b = Rectangle((219, -47.5), 0, 140, linewidth=lw, color=color)
    # 3pt arc - center of arc will be the hoop, arc is 23'9" away from hoop
    # I just played around with the theta values until they lined up with the 
    # threes
    three_arc = Arc((0, 0), 475, 475, theta1=22.5, theta2=157.5, linewidth=lw,
                    color=color)

    # Center Court
    center_outer_arc = Arc((0, 422.5), 120, 120, theta1=180, theta2=0,
                           linewidth=lw, color=color)
    center_inner_arc = Arc((0, 422.5), 40, 40, theta1=180, theta2=0,
                           linewidth=lw, color=color)

    # List of the court elements to be plotted onto the axes
    court_elements = [hoop, backboard, outer_box, inner_box, top_free_throw,
                      bottom_free_throw, restricted, corner_three_a,
                      corner_three_b, three_arc, center_outer_arc,
                      center_inner_arc]

    if outer_lines:
        # Draw the half court line, baseline and side out bound lines
        outer_lines = Rectangle((-250, -47.5), 500, 470, linewidth=lw,
                                color=color, fill=False)
        court_elements.append(outer_lines)

    # Add the court elements onto the axes
    for element in court_elements:
        ax.add_patch(element)

    return ax


In [ ]:
# Shot Graph Comparing the Two Players 

plt.figure(figsize=(12,11))
plt.scatter(harden_shots.LOC_X, harden_shots.LOC_Y, color = "red")
plt.scatter(curry_shots.LOC_X, curry_shots.LOC_Y, color = "blue")
draw_court(outer_lines=True)

plt.xlim(300,-300)
plt.ylim(-100,500)
plt.show()

### Transformation (2) : Average Shots Distance Per Game of Stephen Curry

In [ ]:
# we already have Curry's shot data pulled up from above
curry_shots.head()

In [ ]:
# group data frame for game period first

for r, curry_shots_per_game in curry_shots.groupby('GAME_ID'):
    print("***", r)
    print(curry_shots_per_game.head())

In [ ]:
curry_shots.groupby('GAME_ID')['SHOT_DISTANCE'].mean().plot()